# W207- Applied Machine Learning - Section 1

## Group Project: Random Acts of Pizza

## Group 1: Sartaj Baveja, Tim Spittle, Jay Venkata & Angela Wu

## Overview

Link to Kaggle Competition: https://www.kaggle.com/c/random-acts-of-pizza

This competition contains a dataset of textual requests for pizza from the Reddit community- Random Acts of Pizza (https://www.reddit.com/r/Random_Acts_Of_Pizza/) together with their outcome (successful/unsuccessful) and meta-data. The objective is to build a model that can predict whether or not a post will result in a pizza purchase.

### Objective

The objective of this project is to create an algorithm capable of predicting whether a given post on "Random Acts of Pizza" is likely to be fulfilled based on it's message content and various meta-characteristics. 

## Data Preparation

In [18]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import json
import os
from pprint import pprint
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# SK Learn Libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

This dataset includes 5671 requests between December 8, 2010 and September 29, 2013 (retrieved on September 30, 2013). All requests ask for the same thing: a free pizza. The outcome of each request --- whether its author received a pizza or not --- is known. Meta-data includes information such as: time of the request, activity of the requester, community-age of the requester, etc.  

Each entry in pizza_request_dataset.json corresponds to one request (the first and only request by the requester).

### Fields in each request

| Field Name | Description |
| ---------- | ----------- |
| giver_username_if_known | Reddit username of giver if known, i.e. the person satisfying the request ("N/A" otherwise). |
| in_test_set | Boolean indicating whether this request was part of our test set. |
| number_of_downvotes_of_request_at_retrieval | Number of downvotes at the time the request was collected. |
| number_of_upvotes_of_request_at_retrieval | Number of upvotes at the time the request was collected. |
| post_was_edited | Boolean indicating whether this post was edited (from Reddit). |
| request_id | Identifier of the post on Reddit, e.g. "t3_w5491". |
| request_number_of_comments_at_retrieval | Number of comments for the request at time of retrieval. |
| request_text | Full text of the request. |
| request_text_edit_aware | Edit aware version of "request_text". We use a set of rules to strip edited comments indicating the success of the request such as "EDIT: Thanks /u/foo, the pizza was delicous". |
| request_title | Title of the request. |
| requester_account_age_in_days_at_request | Account age of requester in days at time of request. |
| requester_account_age_in_days_at_retrieval | Account age of requester in days at time of retrieval. |
| requester_days_since_first_post_on_raop_at_request | Number of days between requesters first post on RAOP and this request (zero if requester has never posted before on RAOP). |
| requester_days_since_first_post_on_raop_at_retrieval | Number of days between requesters first post on RAOP and time of retrieval. |
| requester_number_of_comments_at_request | Total number of comments on Reddit by requester at time of request. |
| requester_number_of_comments_at_retrieval | Total number of comments on Reddit by requester at time of retrieval. |
| requester_number_of_comments_in_raop_at_request | Total number of comments in RAOP by requester at time of request. |
| requester_number_of_comments_in_raop_at_retrieval | Total number of comments in RAOP by requester at time of retrieval. |
| requester_number_of_posts_at_request | Total number of posts on Reddit by requester at time of request. |
| requester_number_of_posts_at_retrieval | Total number of posts on Reddit by requester at time of retrieval. |
| requester_number_of_posts_on_raop_at_request | Total number of posts in RAOP by requester at time of request. |
| requester_number_of_posts_on_raop_at_retrieval | Total number of posts in RAOP by requester at time of retrieval. |
| requester_number_of_subreddits_at_request | The number of subreddits in which the author had already posted in at the time of request. |
| requester_received_pizza | Boolean indicating the success of the request, i.e., whether the requester received pizza. |
| requester_subreddits_at_request | The list of subreddits in which the author had already posted in at the time of request. |
| requester_upvotes_minus_downvotes_at_request | Difference of total upvotes and total downvotes of requester at time of request. |
| requester_upvotes_minus_downvotes_at_retrieval | Difference of total upvotes and total downvotes of requester at time of retrieval. |
| requester_upvotes_plus_downvotes_at_request | Sum of total upvotes and total downvotes of requester at time of request. |
| requester_upvotes_plus_downvotes_at_retrieval | Sum of total upvotes and total downvotes of requester at time of retrieval. |
| requester_user_flair | Users on RAOP receive badges (Reddit calls them flairs) which is a small picture next to their username. In our data set the user flair is either None (neither given nor received pizza, N=4282), "shroom" (received pizza, but not given, N=1306), or "PIF" (pizza given after having received, N=83). |
| requester_username | Reddit username of requester. |
| unix_timestamp_of_request | Unix timestamp of request (supposedly in timezone of user, but in most cases it is equal to the UTC timestamp -- which is incorrect since most RAOP users are from the USA). |
| unix_timestamp_of_request_utc | Unit timestamp of request in UTC. |

### Initial Loading

In [21]:
## Load JSON
with open(os.path.join(os.getcwd(), 'train.json')) as org_train_data_file:    
    org_train_data = json.load(org_train_data_file)

## Print the first entry
# pprint(org_train_data[0])

In [20]:
org_train_df_alt = pd.DataFrame.from_records(org_train_data)
org_train_df_alt.head()

giver_username_if_known  number_of_downvotes_of_request_at_retrieval  \
0                     N/A                                            0   
1                     N/A                                            2   
2                     N/A                                            0   
3                     N/A                                            0   
4                     N/A                                            6   

   number_of_upvotes_of_request_at_retrieval post_was_edited request_id  \
0                                          1           False   t3_l25d7   
1                                          5           False   t3_rcb83   
2                                          3           False   t3_lpu5j   
3                                          1            True   t3_mxvj3   
4                                          6           False  t3_1i6486   

   request_number_of_comments_at_retrieval  \
0                                        0   
1                                        0   
2                                        0   
3                                        4   
4                                        5   

                                        request_text  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                             request_text_edit_aware  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                                       request_title  \
0            Request Colorado Springs Help Us Please   
1  [Request] California, No cash and I could use ...   
2  [Request] Hungry couple in Dundee, Scotland wo...   
3  [Request] In Canada (Ontario), just got home f...   
4  [Request] Old friend coming to visit. Would LO...   

   requester_account_age_in_days_at_request              ...                \
0                                  0.000000              ...                 
1                                501.111100              ...                 
2                                  0.000000              ...                 
3                                  6.518438              ...                 
4                                162.063252              ...                 

   requester_received_pizza  \
0                     False   
1                     False   
2                     False   
3                     False   
4                     False   

                     requester_subreddits_at_request  \
0                                                 []   
1  [AskReddit, Eve, IAmA, MontereyBay, RandomKind...   
2                                                 []   
3       [AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]   
4  [GayBrosWeightLoss, RandomActsOfCookies, Rando...   

   requester_upvotes_minus_downvotes_at_request  \
0                                             0   
1                                            34   
2                                             0   
3                                            54   
4                                          1121   

   requester_upvotes_minus_downvotes_at_retrieval  \
0                                               1   
1                                            4258   
2                                               3   
3                                              59   
4                                            1225   

   requester_upvotes_plus_downvotes_at_request  \
0                                            0   
1                                          116   
2                                            0   

In [22]:
## Convert JSON to panda dataframe
org_train_df_raw = pd.io.json.json_normalize(org_train_data)
org_train_df = org_train_df_raw[[
    'giver_username_if_known', 
    'number_of_downvotes_of_request_at_retrieval',
    'number_of_upvotes_of_request_at_retrieval',
    'post_was_edited', 
    'request_id', 
    'request_number_of_comments_at_retrieval',
    'request_text',
    'request_text_edit_aware',
    'request_title',
    'requester_account_age_in_days_at_request',
    'requester_account_age_in_days_at_retrieval',
    'requester_days_since_first_post_on_raop_at_request',
    'requester_days_since_first_post_on_raop_at_retrieval',
    'requester_number_of_comments_at_request',
    'requester_number_of_comments_at_retrieval',
    'requester_number_of_comments_in_raop_at_request',
    'requester_number_of_comments_in_raop_at_retrieval',
    'requester_number_of_posts_at_request',
    'requester_number_of_posts_at_retrieval',
    'requester_number_of_posts_on_raop_at_request',
    'requester_number_of_posts_on_raop_at_retrieval',
    'requester_number_of_subreddits_at_request',
    'requester_received_pizza',
    'requester_subreddits_at_request',
    'requester_upvotes_minus_downvotes_at_request',
    'requester_upvotes_minus_downvotes_at_retrieval',
    'requester_upvotes_plus_downvotes_at_request',
    'requester_upvotes_plus_downvotes_at_retrieval',
    'requester_user_flair',
    'requester_username',
    'unix_timestamp_of_request',
    'unix_timestamp_of_request_utc'
    ]]
org_train_df.head()

giver_username_if_known  number_of_downvotes_of_request_at_retrieval  \
0                     N/A                                            0   
1                     N/A                                            2   
2                     N/A                                            0   
3                     N/A                                            0   
4                     N/A                                            6   

   number_of_upvotes_of_request_at_retrieval post_was_edited request_id  \
0                                          1           False   t3_l25d7   
1                                          5           False   t3_rcb83   
2                                          3           False   t3_lpu5j   
3                                          1            True   t3_mxvj3   
4                                          6           False  t3_1i6486   

   request_number_of_comments_at_retrieval  \
0                                        0   
1                                        0   
2                                        0   
3                                        4   
4                                        5   

                                        request_text  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                             request_text_edit_aware  \
0  Hi I am in need of food for my 4 children we a...   
1  I spent the last money I had on gas today. Im ...   
2  My girlfriend decided it would be a good idea ...   
3  It's cold, I'n hungry, and to be completely ho...   
4  hey guys:\n I love this sub. I think it's grea...   

                                       request_title  \
0            Request Colorado Springs Help Us Please   
1  [Request] California, No cash and I could use ...   
2  [Request] Hungry couple in Dundee, Scotland wo...   
3  [Request] In Canada (Ontario), just got home f...   
4  [Request] Old friend coming to visit. Would LO...   

   requester_account_age_in_days_at_request              ...                \
0                                  0.000000              ...                 
1                                501.111100              ...                 
2                                  0.000000              ...                 
3                                  6.518438              ...                 
4                                162.063252              ...                 

   requester_received_pizza  \
0                     False   
1                     False   
2                     False   
3                     False   
4                     False   

                     requester_subreddits_at_request  \
0                                                 []   
1  [AskReddit, Eve, IAmA, MontereyBay, RandomKind...   
2                                                 []   
3       [AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]   
4  [GayBrosWeightLoss, RandomActsOfCookies, Rando...   

   requester_upvotes_minus_downvotes_at_request  \
0                                             0   
1                                            34   
2                                             0   
3                                            54   
4                                          1121   

   requester_upvotes_minus_downvotes_at_retrieval  \
0                                               1   
1                                            4258   
2                                               3   
3                                              59   
4                                            1225   

   requester_upvotes_plus_downvotes_at_request  \
0                                            0   
1                                          116   
2                                            0   

In [19]:
# Split the original training data into training and development
train_df, dev_df = train_test_split(org_train_df, test_size = 0.4)

### Data Cleaning

- De-duplicate
- Remove edited posts? (accounting for those we know were edited with indications of success (see: "request_text_edit_aware")
- etc

### Features of Interest

#### Meta-Data Features

_General Note_ "at retrieval" statistics are not good features: unclear when activity happened (pre-post fulfillment)  

**Time**  
Basic feature - logically there are probably times when more people are active and willing to fulfill requests

**Requester Profile**    
Trying to make distinction between: 
- _Primary_ profile features (i.e. things an average reader of RAOP may see)
    - RAOP flair
    - New/Throwaway account
    - Multi-requester
- _Secondary_ features (i.e. extended detail about activity on Reddit that a potential "Giver" might not see, and therefore it wouldn't influence their likelihood of fulfilling the request or not)  
    - Comments
    - Posts
    - Sub-Reddits
    - Upvotes/Downvotes

**Community Effects (Secondary)**  
- Exogenous - Reactions to a request (upvotes/downvotes/comments) may correlated with success/fulfillment but we don't know this info at time of the request nor do we know if this activity happened before/after the request was fulfilled. 
- May be good for a sensitivity analysis, but probably not base model.  

#### Text Analysis Plan

### Feature Engineering

## Models

## Results